In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import codecs
import numpy as np
import nltk
import pycrfsuite
from bs4 import BeautifulSoup as bs
from bs4.element import Tag
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_validate
import re
import random
import pandas as pd
from itertools import chain
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

import numpy as n
import tensorflow
from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from tensorflow.keras.utils import plot_model

from tensorflow.python.keras.preprocessing.text import Tokenizer, one_hot
from keras.optimizers import SGD

from numpy.random import seed
seed(1)
tensorflow.random.set_seed(2)

from keras_contrib.layers import CRF
# from tensorflow_addons.layers import CRF

import numpy as np
import tensorflow
from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from tensorflow.keras.utils import plot_model


from numpy.random import seed
seed(1)
tensorflow.random.set_seed(2)

import os
from os import listdir
from os.path import isfile, join
import pickle

from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

import gensim

Using TensorFlow backend.


In [3]:
with open('complex_rules_iob_input.pkl', 'rb') as f:
    dt_list = pickle.load(f)

In [4]:
with open('word_embeddings_dict.pkl', 'rb') as f:
    word_embeddings_dictionary = pickle.load(f)
    print(len(word_embeddings_dictionary.values()))

5664


In [5]:
new_df = []
for i in range(0, len(dt_list)):
    cur_list = dt_list[i]
    for j in range(0, len(cur_list)):
        cur_item = cur_list[j]
        if j == 0:
            new_df += [['Sentence: ' + str(i+1), cur_item[0], cur_item[1], cur_item[2]]]
        else:
            new_df += [[np.nan, cur_item[0], cur_item[1], cur_item[2]]]
#         new_df += [['Sentence: ' + str(i+1), cur_item[0], cur_item[1], cur_item[2]]]

# print(new_df)

In [6]:
data = pd.DataFrame(new_df, columns=['Sentence #', 'Word', 'POS', 'Tag'])

In [7]:
data.head(80)

,Sentence #,Word,POS,Tag
0,Sentence: 1,Frozen,JJ,O
1,NaN,sections,NNS,O
2,NaN,of,IN,O
3,NaN,various,JJ,O
4,NaN,green,JJ,O
...,...,...,...,...
75,NaN,different,JJ,O
76,NaN,treatments,NNS,O
77,NaN,on,IN,O
78,NaN,cell-mediated,JJ,O


In [8]:
def get_dict_map(data, token_or_tag):
    tok2idx = {}
    idx2tok = {}
    
    if token_or_tag == 'token':
        vocab = list(set(data['Word'].to_list()))
    else:
        vocab = list(set(data['Tag'].to_list()))
    
    idx2tok = {idx:tok for  idx, tok in enumerate(vocab)}
    tok2idx = {tok:idx for  idx, tok in enumerate(vocab)}
#     tok2idx = {tok: np.array(word_embeddings_dictionary[tok.lower()]) for tok in vocab if tok.lower() in word_embeddings_dictionary}
    
    return tok2idx, idx2tok

In [9]:
token2idx, idx2token = get_dict_map(data, 'token')
# tag2idx, idx2tag = get_dict_map(data, 'tag')

# tag2idx = {None: 0, "O": 1, "I-method": 2, "B-method": 3}
# idx2tag = {0: None, 1: "O", 2: "I-method", 3: "B-method"}

tag2idx = {"O": 0, "I-method": 1, "B-method": 2}
idx2tag = {0: "O", 1: "I-method", 2: "B-method"}

In [10]:
data['Word_idx'] = data['Word'].map(token2idx)
data['Tag_idx'] = data['Tag'].map(tag2idx)

# data = data[data['Word_idx'].notna()]
data.head()

,Sentence #,Word,POS,Tag,Word_idx,Tag_idx
0,Sentence: 1,Frozen,JJ,O,5216,0
1,NaN,sections,NNS,O,4266,0
2,NaN,of,IN,O,1313,0
3,NaN,various,JJ,O,2318,0
4,NaN,green,JJ,O,4770,0


In [11]:
def f(row, model):
    try:
        return model[row]
    except Exception as e:
        return None

In [12]:
# Fill na
data_fillna = data.fillna(method='ffill', axis=0)
# data_fillna = data
# Groupby and collect columns
data_group = data_fillna.groupby(
['Sentence #'],as_index=False
)['Word', 'POS', 'Tag', 'Word_idx', 'Tag_idx'].agg(lambda x: list(x))
# Visualise data
data_group.head()

<ipython-input-12-9ba350958316>:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  data_group = data_fillna.groupby(


,Sentence #,Word,POS,Tag,Word_idx,Tag_idx
0,Sentence: 1,"[Frozen, sections, of, various, green, fluores...","[JJ, NNS, IN, JJ, JJ, JJ, NN, JJ, NN, NNS, VBD...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-m...","[5216, 4266, 1313, 2318, 4770, 1583, 5770, 489...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, ..."
1,Sentence: 10,"[To, develop, a, reproducible, and, accurate, ...","[TO, VB, DT, JJ, CC, JJ, NN, IN, NN, NN, NN, I...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[4966, 5775, 6180, 2642, 3738, 3254, 4853, 131...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,Sentence: 100,"[To, discard, tags, that, are, likely, to, hav...","[TO, VB, NNS, WDT, VBP, JJ, TO, VB, VBN, VBN, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[4966, 4740, 1352, 3178, 2200, 2953, 4052, 400...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,Sentence: 1000,"[The, objectives, of, this, study, were, :, 1,...","[DT, NNS, IN, DT, NN, VBD, :, LS, -RRB-, TO, V...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[6221, 4431, 1313, 949, 110, 2116, 1082, 1061,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,Sentence: 1001,"[More, recently, ,, surgeons, tried, to, refin...","[RBR, RB, ,, NNS, VBD, TO, VB, DT, NN, NN, IN,...","[O, O, O, O, O, O, O, O, B-method, I-method, O...","[2063, 5381, 3683, 3793, 3337, 4052, 5015, 949...","[0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, ..."


In [13]:
def get_pad_train_test_val(data_group, data):
    n_token = len(list(set(data['Word'].to_list())))
    n_tag = len(list(set(data['Tag'].to_list())))
    
    tokens = data_group['Word_idx'].tolist()
    
    maxlen = max([len(s) for s in tokens])
#     pad_tokens = pad_sequences(tokens, maxlen=maxlen, dtype='int32', padding='post', value= n_token - 1)
#     pad_tokens = pad_sequences(tokens, maxlen=maxlen, dtype='int32', padding='post', value=0)
#     pad_tokens = pad_sequences(tokens, maxlen=maxlen, dtype='int32', padding='post', truncating='post', value=-1)
    pad_tokens = pad_sequences(tokens, maxlen=maxlen)
#     pad_tokens = tensorflow.convert_to_tensor(tokens)
#     pad_tokens = np.array(tokens)
    
    tags = data_group['Tag_idx'].tolist()
#     pad_tags = pad_sequences(tags, maxlen=maxlen, dtype='int32', padding='post', value= tag2idx["O"])
#     pad_tags = pad_sequences(tags, maxlen=maxlen, dtype='int32', padding='post', value=0)
#     pad_tags = pad_sequences(tags, maxlen=maxlen, dtype='int32', padding='post', truncating='post', value=-1)
    pad_tags = pad_sequences(tags, maxlen=maxlen)
#     pad_tags = tensorflow.convert_to_tensor(tags)
#     pad_tags = np.array(tags)
    
    n_tags = len(tag2idx)
    pad_tags = [to_categorical(i, num_classes=n_tags) for i in pad_tags]
    
    
    train_tokens, test_tokens, train_tags, test_tags = train_test_split(pad_tokens, pad_tags, test_size=0.1, train_size=0.9, random_state=2020)

    print(
        'train_tokens length:', len(train_tokens),
        '\ntest_tokens length:', len(test_tokens),
        '\ntrain_tags:', len(train_tags),
        '\ntest_tags:', len(test_tags)
    )
    
    return train_tokens, test_tokens, train_tags, test_tags

train_tokens, test_tokens, train_tags, test_tags = get_pad_train_test_val(data_group, data)

train_tokens length: 1204 
test_tokens length: 134 
train_tags: 1204 
test_tags: 134


In [14]:
input_dim = len(list(set(data['Word'].to_list())))+1
output_dim = 32
input_length = max([len(s) for s in data_group['Word_idx'].tolist()])
n_tags = len(tag2idx)
print('input_dim: ', input_dim, '\noutput_dim: ', output_dim, '\ninput_length: ', input_length, '\nn_tags: ', n_tags)

input_dim:  6254 
output_dim:  32 
input_length:  105 
n_tags:  3


In [15]:
vocab = list(set(data['Word'].to_list()))
embedding_matrix = np.zeros((input_dim-1, 200))
for i in range(len(vocab)):
    word = vocab[i]
    embedding_vector = word_embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [16]:
print(embedding_matrix)

[[-0.010897   0.35497    0.59805   ...  0.0093769 -0.37428   -0.68876  ]
 [ 0.         0.         0.        ...  0.         0.         0.       ]
 [ 0.         0.         0.        ...  0.         0.         0.       ]
 ...
 [ 0.38194    1.2685    -0.63251   ...  0.13338    0.19931    0.17543  ]
 [ 0.78099    1.0193     0.01486   ...  0.36567   -0.39012    0.20763  ]
 [ 0.         0.         0.        ...  0.         0.         0.       ]]


In [17]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(1)

def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        self.word_embeds.weight.data.copy_(torch.from_numpy(embedding_matrix))
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2),
                torch.randn(2, 1, self.hidden_dim // 2))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

In [18]:
X_train, X_test, y_train, y_test = train_test_split(data_group['Word'], data_group['Tag'], test_size=0.33, random_state=42)

In [19]:
torch.from_numpy(embedding_matrix).shape

torch.Size([6253, 200])

In [20]:
START_TAG = "<START>"
STOP_TAG = "<STOP>"
EMBEDDING_DIM = 200
HIDDEN_DIM = 4

# Make up some training data
# training_data = [(
#     "the wall street journal reported today that apple corporation made money".split(),
#     "B I I I O O O B I O O".split()
# ), (
#     "georgia tech is a university in georgia".split(),
#     "B I O O O O B".split()
# )]

# word_to_ix = {}
# for sentence, tags in training_data:
#     for word in sentence:
#         if word not in word_to_ix:
#             word_to_ix[word] = len(word_to_ix)

# tag_to_ix = {"B": 0, "I": 1, "O": 2, START_TAG: 3, STOP_TAG: 4}
tag2idx[START_TAG] = tag2idx['B-method']+1
tag2idx[STOP_TAG] = tag2idx['B-method']+2

# print(tag2idx)

model = BiLSTM_CRF(len(token2idx), tag2idx, EMBEDDING_DIM, HIDDEN_DIM)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)



# Check predictions before training
with torch.no_grad():
    precheck_sent = prepare_sequence(X_train[0], token2idx)
    precheck_tags = torch.tensor([tag2idx[t] for t in y_train[0]], dtype=torch.long)
    print(model(precheck_sent))

# Make sure prepare_sequence from earlier in the LSTM section is loaded
for epoch in range(3):  # again, normally you would NOT do 300 epochs, it is toy data
#     print("Epoch: ", epoch)
#     i = 0
    for sentence, tags in zip(X_train, y_train):
#         i+=1
#         print(str(i), ': ', ' '.join(sentence), '\n' + str(i), ': ', ' '.join(tags), '\n\n')
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = prepare_sequence(sentence, token2idx)
        targets = torch.tensor([tag2idx[t] for t in tags], dtype=torch.long)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

(tensor(19.1609), [1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1])


/home/wkalim/.local/lib/python3.8/site-packages/torch/autograd/__init__.py:130: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  Variable._execution_engine.run_backward(


In [21]:
# Check predictions after training
with torch.no_grad():
    precheck_sent = prepare_sequence(X_train[0], token2idx)
    print([idx2tag[x] for x in model(precheck_sent)[1]])
# We got it!

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-method', 'I-method', 'O', 'O', 'O', 'O', 'O', 'O']


In [22]:
true_all = np.argmax(test_tags, -1)

true_all_tags = [[idx2tag[idx] for idx in s] for s in true_all]

In [23]:
p_all = []
with torch.no_grad():
    for i in X_test:
        p_all.append(np.array(model(prepare_sequence(i, token2idx))[1]))

p_all_tags = [[idx2tag[idx] for idx in s] for s in p_all] # ['B-gpe', 'O', 'O', 'O']
# print(p_all_tags)

In [24]:
# print("{:15}||{:5}||{}".format("Word", "True", "Pred"))
# print(30 * "=")
print("{: >20} {: >20} {: >20} {: >20} {: >20} ".format("Word", "True", "Pred", "New", "Missed"))
print(110 * "=")

for w, t, pred in zip(X_test, y_test, p_all_tags):
    for i in range(len(w)):
        if w != 0:
#             print("{:15}: {:5} {}".format(w[i], t[i], pred[i]))
            print("{: >20} {: >20} {: >20} {: >20} {: >20} ".format(w[i], t[i], pred[i], 1 if t[i] != pred[i] and t[i] == 'O' else "", 1 if t[i] != pred[i] and pred[i] == 'O' else ""))

    print(110 * "=")

                Word                 True                 Pred                  New               Missed 
             Another                    O             B-method                    1                      
        transfection             B-method             I-method                                           
              method             I-method             I-method                                           
                   ,                    O                    O                                           
     electroporation                    O                    O                                           
                   ,                    O                    O                                           
                  is                    O                    O                                           
                  an                    O                    O                                           
        experimental                    O     

                   .                    O                    O                                           
                eQTL             B-method             B-method                                           
            analysis             I-method             I-method                                           
                 was                    O                    O                                           
             applied                    O                    O                                           
                  to                    O                    O                                           
               mouse                    O                    O                                           
             tissues                    O                    O                                           
                from                    O                    O                                           
                  an                    O     

            combines                    O                    O                                           
                  an                    O                    O                                           
                 HMM             B-method                    O                                         1 
            approach             I-method                    O                                         1 
                   ,                    O                    O                                           
                   a                    O                    O                                           
         progressive                    O             B-method                    1                      
           alignment                    O             I-method                    1                      
           algorithm                    O             I-method                    1                      
                   ,                    O     

                 the                    O                    O                                           
              target                    O                    O                                           
                gene                    O                    O                                           
                  is                    O                    O                                           
        standardized                    O                    O                                           
                  by                    O                    O                                           
                   a                    O                    O                                           
       non-regulated                    O                    O                                           
           reference                    O                    O                                           
                gene                    O     

              fitted                    O                    O                                           
                  by                    O                    O                                           
            isotonic                    O                    O                                           
          regression                    O                    O                                           
                 can                    O                    O                                           
                  be                    O                    O                                           
                used                    O                    O                                           
                  to                    O                    O                                           
            estimate                    O                    O                                           
           threshold                    O     

            evaluate                    O                    O                                           
            evidence                    O                    O                                           
                  of                    O                    O                                           
                  LD                    O                    O                                           
                  in                    O                    O                                           
             general                    O                    O                                           
            pedigree                    O                    O                                           
                data                    O                    O                                           
                   .                    O                    O                                           
                 The                    O     

                   ,                    O                    O                                           
                 LPS                    O                    O                                           
                   ,                    O                    O                                           
               class                    O                    O                                           
                   5                    O                    O                                           
                 and                    O                    O                                           
      cross-reactive                    O                    O                                           
         specificity                    O                    O                                           
                  of                    O                    O                                           
                Mabs                    O     

                   ,                    O                    O                                           
                 and                    O                    O                                           
           treatment                    O                    O                                           
              groups                    O                    O                                           
            compared                    O                    O                                           
               using                    O                    O                                           
                 Log                    O                    O                                           
                rank                    O                    O                                           
               tests                    O                    O                                           
                   .                    O     

                loss                    O                    O                                           
                  of                    O                    O                                           
               donor                    O                    O                                           
        fluorescence                    O                    O                                           
                   ,                    O                    O                                           
                  is                    O                    O                                           
            critical                    O                    O                                           
                 for                    O                    O                                           
         calculating                    O                    O                                           
                FRET                    O     

In [25]:
idx2word = {value: key for key, value in token2idx.items()}
idx2tag = {value: key for key, value in tag2idx.items()}

In [26]:
true_all = np.argmax(test_tags, -1)

true_all_tags = [[idx2tag[idx] for idx in s] for s in true_all]

In [27]:
print("{:15}||{:5}||{}".format("Word", "True", "Pred"))
print(30 * "=")

for i in range(len(test_tokens)):
    for w, t, pred in zip(test_tokens[i], true_all_tags[i], p_all_tags[i]):
#         print(w)
        if w != 0:
            print("{:15}: {:5} {}".format(idx2word[w], t, pred))
#         print(30 * "=")

Word           ||True ||Pred
Our            : O     O
network        : B-method O
model          : I-method O
provided       : O     O
a              : O     O
good           : O     O
way            : O     O
to             : O     O
solve          : O     O
the            : O     O
problem        : O     O
by             : O     O
assigning      : O     O
variances      : O     O
to             : O     O
the            : O     O
predictions    : O     O
to             : O     O
obtain         : O     O
The            : O     O
validation     : O     O
process        : O     O
followed       : O     O
a              : O     O
slightly       : O     O
modified       : O     O
version        : O     O


In [28]:
print("{:15}||{:5}||{}".format("Word", "True", "Pred"))
print(30 * "=")

ith_sentence = 0

for w, t, pred in zip(test_tokens[ith_sentence], true_all_tags[ith_sentence], p_all_tags[ith_sentence]):
    if w != 0:
        print("{:15}: {:5} {}".format(idx2word[w], t, pred))

Word           ||True ||Pred


In [29]:
for i, true in enumerate(true_all_tags):
    length = len(true)
    p_all_tags[i] = p_all_tags[i][:length]

In [30]:
# p_all_tags = [[x.replace('<pad>', 'O') for x in s] for s in p_all_tags]
p_all_tags = [[x for x in s] for s in p_all_tags]

In [31]:
y_true = [[x for x in y if x != None] for y in y_test]
y_pred = [[x for x in y if x != None] for y in p_all_tags]

In [32]:

y_true = []
y_pred = []

for x, i, j in zip(X_test, y_test, p_all_tags):
    if len(i) != len(j):
        print(len(i), len(j))
        print(' '.join([idx2word[idx] for idx in x]))
        print(i)
        print(j)
        print('\n')
    else:
        y_true.append(i)
        y_pred.append(j)
#     print(len(i), len(j))


In [33]:
print(classification_report([x for y in y_true for x in y], [x for y in y_pred for x in y]))

              precision    recall  f1-score   support

    B-method       0.72      0.74      0.73       431
    I-method       0.78      0.83      0.81       850
           O       0.98      0.97      0.98     10578

    accuracy                           0.96     11859
   macro avg       0.83      0.85      0.84     11859
weighted avg       0.96      0.96      0.96     11859

